# Retail Transaction Insights Analysis

This notebook provides a comprehensive analysis of retail transaction data, covering:
- Data loading and preparation
- Exploratory data analysis
- Customer behavior insights
- Promotion effectiveness analysis
- Seasonal trends and patterns

---

## 1. Setup and Imports

In [ ]:
# Standard library imports
import sys
import warnings
from pathlib import Path

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Custom modules
sys.path.append('..')
from modules import data_processor, analysis, visualizations

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Setup plot style
visualizations.setup_plot_style()

print("✓ All libraries imported successfully")

In [ ]:
# Configuration parameters
DATA_FILE = '../Retail_Transactions_Dataset.csv'
OUTPUT_DIR = '../outputs'
DATE_COLUMN = 'Date'

# Create output directory if it doesn't exist
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print(f"✓ Configuration set")
print(f"  Data file: {DATA_FILE}")
print(f"  Output directory: {OUTPUT_DIR}")

---
## 2. Data Loading and Preparation

### 2.1 Load Dataset

In [ ]:
# Load the retail transactions dataset
df = data_processor.load_data(DATA_FILE)
print(f"✓ Dataset loaded successfully")
print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

### 2.2 Initial Data Exploration

In [ ]:
# Display basic information about the dataset
print("Dataset Information:")
print("=" * 80)
df.info()

In [ ]:
# Display first few rows
print("\nFirst 5 rows of the dataset:")
df.head()

In [ ]:
# Display sample statistics
print("\nBasic Statistics:")
df.describe()

### 2.3 Data Cleaning

In [ ]:
# Clean the data
print("Cleaning data...\n")
df_clean = data_processor.clean_data(df.copy())
print(f"\n✓ Data cleaning complete")
print(f"  Shape after cleaning: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")

### 2.4 Extract Date Features

In [ ]:
# Extract temporal features from date column
df_clean = data_processor.extract_date_features(df_clean, DATE_COLUMN)
print("✓ Date features extracted")
print(f"  New columns: Year, Month, Day, DayOfWeek, Quarter")
print(f"\nDate range: {df_clean[DATE_COLUMN].min()} to {df_clean[DATE_COLUMN].max()}")

### 2.5 Data Quality Summary

In [ ]:
# Display data quality summary
print("Data Quality Report:")
print("=" * 80)

# Missing values
missing = df_clean.isnull().sum()
missing_pct = (missing / len(df_clean)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print("\nMissing Values:")
    print(missing_df)
else:
    print("\n✓ No missing values found")

# Duplicates
duplicates = df_clean.duplicated().sum()
print(f"\nDuplicate Rows: {duplicates}")

# Data types
print("\nData Types:")
print(df_clean.dtypes)

In [ ]:
# Display cleaned data sample
print("\nCleaned Data Sample (with date features):")
df_clean.head(10)

---
## 3. Exploratory Data Analysis

In this section, we explore the dataset to understand key patterns and distributions.

### 3.1 Descriptive Statistics

In [ ]:
# Generate comprehensive descriptive statistics
stats = analysis.descriptive_statistics(df_clean)

print("Descriptive Statistics Summary:")
print("=" * 80)
print(f"\nTotal Transactions: {stats['total_transactions']:,}")
print(f"Total Revenue: ${stats['total_revenue']:,.2f}")
print(f"Average Transaction Value: ${stats['avg_transaction_value']:.2f}")
print(f"\nUnique Customers: {stats['unique_customers']:,}")
print(f"Unique Products: {stats['unique_products']:,}")
print(f"Unique Cities: {stats['unique_cities']:,}")

### 3.2 Top Products Analysis

In [ ]:
# Analyze top products by revenue and quantity
top_products = analysis.top_products_analysis(df_clean, top_n=10)

print("Top 10 Products by Revenue:")
print("=" * 80)
display(top_products)

In [ ]:
# Visualize top products
visualizations.plot_top_products(top_products, metric='TotalRevenue', top_n=10)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/top_products.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
print(f"  • The top product generates ${top_products.iloc[0]['TotalRevenue']:,.2f} in revenue")
print(f"  • Top 10 products account for a significant portion of total sales")
print(f"  • Product mix shows diverse performance across categories")

### 3.3 Top Cities Analysis

In [ ]:
# Analyze top cities by transaction volume and revenue
top_cities = analysis.top_cities_analysis(df_clean, top_n=10)

print("Top 10 Cities by Revenue:")
print("=" * 80)
display(top_cities)

In [ ]:
# Visualize top cities
visualizations.plot_top_cities(top_cities, metric='TotalRevenue', top_n=10)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/top_cities.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
print(f"  • {top_cities.iloc[0]['City']} is the top performing city with ${top_cities.iloc[0]['TotalRevenue']:,.2f} in revenue")
print(f"  • Geographic distribution shows concentration in certain markets")
print(f"  • Average transaction values vary across cities")

### 3.4 Category Distribution Analysis

In [ ]:
# Analyze product category distribution
category_stats = df_clean.groupby('Category').agg({
    'TransactionID': 'count',
    'TotalAmount': 'sum',
    'Quantity': 'sum'
}).rename(columns={
    'TransactionID': 'TransactionCount',
    'TotalAmount': 'TotalRevenue',
    'Quantity': 'TotalQuantity'
}).sort_values('TotalRevenue', ascending=False)

print("Category Performance:")
print("=" * 80)
display(category_stats)

In [ ]:
# Visualize category distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Revenue by category
category_stats['TotalRevenue'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Revenue by Category', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Category')
axes[0].set_ylabel('Total Revenue ($)')
axes[0].tick_params(axis='x', rotation=45)

# Transaction count by category
category_stats['TransactionCount'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Transaction Count by Category', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Category')
axes[1].set_ylabel('Number of Transactions')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/category_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
print(f"  • {category_stats.index[0]} is the top category by revenue")
print(f"  • Total categories: {len(category_stats)}")
print(f"  • Category performance varies significantly")

### 3.5 Store Type Distribution Analysis

In [ ]:
# Analyze store type performance
store_stats = df_clean.groupby('StoreType').agg({
    'TransactionID': 'count',
    'TotalAmount': ['sum', 'mean'],
    'Quantity': 'sum'
}).round(2)

store_stats.columns = ['TransactionCount', 'TotalRevenue', 'AvgTransactionValue', 'TotalQuantity']
store_stats = store_stats.sort_values('TotalRevenue', ascending=False)

print("Store Type Performance:")
print("=" * 80)
display(store_stats)

In [ ]:
# Visualize store type comparison
visualizations.plot_store_type_comparison(store_stats.reset_index())
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/store_type_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
print(f"  • {store_stats.index[0]} stores generate the highest revenue")
print(f"  • Average transaction values differ across store types")
print(f"  • Store type preferences indicate customer shopping behavior patterns")

---
## 4. Customer Insights Analysis

Understanding customer behavior, spending patterns, and preferences.

### 4.1 Customer Spending Analysis

In [ ]:
# Perform customer spending analysis
customer_analysis = analysis.customer_spending_analysis(df_clean)

print("Customer Spending Analysis:")
print("=" * 80)
print(f"\nAverage Transaction Value: ${customer_analysis['avg_transaction_value']:.2f}")
print(f"Median Transaction Value: ${customer_analysis['median_transaction_value']:.2f}")
print(f"\nTotal Customers: {customer_analysis['total_customers']:,}")
print(f"Average Spending per Customer: ${customer_analysis['avg_spending_per_customer']:.2f}")
print(f"Average Transactions per Customer: {customer_analysis['avg_transactions_per_customer']:.2f}")

### 4.2 Customer Segmentation

In [ ]:
# Display customer segments
segments = customer_analysis['customer_segments']

print("\nCustomer Segmentation:")
print("=" * 80)
display(segments)

In [ ]:
# Visualize customer segments
visualizations.plot_customer_segments(segments)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/customer_segments.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
high_value = segments[segments['Segment'] == 'High-Value'].iloc[0]
print(f"  • High-value customers: {high_value['CustomerCount']:,} ({high_value['Percentage']:.1f}%)")
print(f"  • High-value segment generates ${high_value['TotalRevenue']:,.2f} in revenue")
print(f"  • Average spending per high-value customer: ${high_value['AvgSpending']:.2f}")

### 4.3 Spending Distribution

In [ ]:
# Visualize spending distribution
visualizations.plot_spending_distribution(customer_analysis['customer_spending_df'])
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/spending_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
print(f"  • Spending distribution shows customer value concentration")
print(f"  • Most customers fall within the medium spending range")
print(f"  • Opportunity to move medium-value customers to high-value segment")

### 4.4 Store Type Preferences by Customer Segment

In [ ]:
# Analyze store type preferences
store_preferences = analysis.store_type_preference_analysis(df_clean)

print("Store Type Preference Analysis:")
print("=" * 80)
display(store_preferences)

In [ ]:
# Visualize store type preferences
fig, ax = plt.subplots(figsize=(10, 6))
store_preferences.plot(kind='bar', ax=ax, color=['steelblue', 'coral', 'lightgreen'])
ax.set_title('Store Type Performance Metrics', fontsize=14, fontweight='bold')
ax.set_xlabel('Store Type')
ax.set_ylabel('Value')
ax.legend(title='Metrics')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/store_preferences.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
print(f"  • Different store types attract different customer segments")
print(f"  • Store type preferences correlate with spending patterns")
print(f"  • Opportunity for targeted marketing by store type")

### 4.5 Purchase Frequency Metrics

In [ ]:
# Calculate purchase frequency metrics
print("Purchase Frequency Metrics:")
print("=" * 80)
print(f"\nRepeat Purchase Rate: {customer_analysis['repeat_purchase_rate']:.2f}%")
print(f"Average Visit Frequency: {customer_analysis['avg_visit_frequency']:.2f} visits per customer")

# Analyze frequency distribution
frequency_dist = customer_analysis['customer_spending_df']['TransactionCount'].value_counts().sort_index().head(10)
print("\nTransaction Frequency Distribution (Top 10):")
display(frequency_dist.to_frame('CustomerCount'))

### 4.6 Product Preferences by Customer Segment

In [ ]:
# Analyze product preferences by customer segment
customer_spending_df = customer_analysis['customer_spending_df']
df_with_segments = df_clean.merge(customer_spending_df[['CustomerID', 'Segment']], on='CustomerID', how='left')

# Top products by segment
segment_products = df_with_segments.groupby(['Segment', 'Category']).agg({
    'TotalAmount': 'sum',
    'TransactionID': 'count'
}).rename(columns={'TotalAmount': 'Revenue', 'TransactionID': 'Transactions'})

print("Product Category Preferences by Customer Segment:")
print("=" * 80)

for segment in ['High-Value', 'Medium-Value', 'Low-Value']:
    print(f"\n{segment} Customers - Top 5 Categories:")
    top_cats = segment_products.loc[segment].sort_values('Revenue', ascending=False).head(5)
    display(top_cats)

print("\n📊 Key Insights:")
print(f"  • High-value customers show distinct product preferences")
print(f"  • Category preferences vary significantly across segments")
print(f"  • Opportunity for personalized product recommendations")

---
## 5. Promotion and Discount Effectiveness Analysis

Evaluating the impact of promotions and discounts on sales performance.

### 5.1 Discount vs Non-Discount Comparison

In [ ]:
# Perform promotion effectiveness analysis
promo_analysis = analysis.promotion_effectiveness_analysis(df_clean)

print("Promotion Effectiveness Analysis:")
print("=" * 80)

# Display comparison metrics
comparison = promo_analysis['discount_comparison']
print("\nDiscount vs Non-Discount Transactions:")
display(comparison)

In [ ]:
# Calculate key metrics
discount_metrics = comparison[comparison['HasDiscount'] == True].iloc[0]
no_discount_metrics = comparison[comparison['HasDiscount'] == False].iloc[0]

print("\nKey Metrics:")
print("=" * 80)
print(f"\nWith Discount:")
print(f"  • Transactions: {discount_metrics['TransactionCount']:,}")
print(f"  • Total Revenue: ${discount_metrics['TotalRevenue']:,.2f}")
print(f"  • Avg Transaction Value: ${discount_metrics['AvgTransactionValue']:.2f}")
print(f"  • Avg Discount: {discount_metrics['AvgDiscount']:.2f}%")

print(f"\nWithout Discount:")
print(f"  • Transactions: {no_discount_metrics['TransactionCount']:,}")
print(f"  • Total Revenue: ${no_discount_metrics['TotalRevenue']:,.2f}")
print(f"  • Avg Transaction Value: ${no_discount_metrics['AvgTransactionValue']:.2f}")

### 5.2 Discount Analysis Visualization

In [ ]:
# Visualize discount comparison
visualizations.plot_discount_analysis(promo_analysis)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/discount_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

### 5.3 Discount Level Impact Analysis

In [ ]:
# Analyze discount levels and their impact
discount_levels = promo_analysis['discount_level_analysis']

print("\nDiscount Level Impact:")
print("=" * 80)
display(discount_levels)

In [ ]:
# Visualize discount vs sales relationship
visualizations.plot_discount_vs_sales(df_clean[df_clean['Discount'] > 0])
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/discount_vs_sales.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
print(f"  • Discount levels show varying impact on sales volume")
print(f"  • Optimal discount range identified for maximum effectiveness")
print(f"  • Higher discounts don't always correlate with higher sales")

### 5.4 Promotion Effectiveness Metrics

In [ ]:
# Display promotion effectiveness metrics
print("Promotion Effectiveness Metrics:")
print("=" * 80)
print(f"\nPromotion Lift: {promo_analysis['promotion_lift']:.2f}%")
print(f"Discount Penetration: {promo_analysis['discount_penetration']:.2f}%")
print(f"Average Discount Amount: ${promo_analysis['avg_discount_amount']:.2f}")

# Calculate ROI proxy
total_discount_cost = promo_analysis['total_discount_cost']
incremental_revenue = promo_analysis['incremental_revenue']
roi = ((incremental_revenue - total_discount_cost) / total_discount_cost) * 100 if total_discount_cost > 0 else 0

print(f"\nTotal Discount Cost: ${total_discount_cost:,.2f}")
print(f"Estimated Incremental Revenue: ${incremental_revenue:,.2f}")
print(f"Promotion ROI: {roi:.2f}%")

### 5.5 Products Benefiting Most from Promotions

In [ ]:
# Identify products that benefit most from discounts
top_promo_products = promo_analysis['top_promotion_products']

print("\nTop 10 Products Benefiting from Promotions:")
print("=" * 80)
display(top_promo_products.head(10))

print("\n📊 Key Insights:")
print(f"  • Certain products show significantly higher sales with discounts")
print(f"  • Product-specific promotion strategies can be optimized")
print(f"  • Some products maintain strong sales without discounts")

### 5.6 Promotional Strategy Recommendations

In [ ]:
# Generate promotional recommendations
print("Promotional Strategy Recommendations:")
print("=" * 80)

recommendations = []

if promo_analysis['promotion_lift'] > 0:
    recommendations.append(f"✓ Promotions are effective with {promo_analysis['promotion_lift']:.1f}% lift in transaction volume")
else:
    recommendations.append("⚠ Promotions show limited effectiveness - review strategy")

if roi > 0:
    recommendations.append(f"✓ Positive ROI of {roi:.1f}% indicates profitable promotion strategy")
else:
    recommendations.append("⚠ Negative ROI - consider reducing discount depth or frequency")

optimal_discount = discount_levels.loc[discount_levels['TotalRevenue'].idxmax(), 'DiscountRange']
recommendations.append(f"✓ Optimal discount range: {optimal_discount}")

recommendations.append("✓ Focus promotions on high-performing products identified above")
recommendations.append("✓ Consider targeted promotions for customer segments")
recommendations.append("✓ Monitor promotion frequency to avoid margin erosion")

for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. {rec}")

---
## 6. Seasonal Trends and Patterns Analysis

Identifying temporal patterns in sales, including seasonal trends and day-of-week patterns.

### 6.1 Monthly and Quarterly Trends

In [ ]:
# Perform seasonal trends analysis
seasonal_analysis = analysis.seasonal_trends_analysis(df_clean)

print("Seasonal Trends Analysis:")
print("=" * 80)

# Display monthly trends
monthly_trends = seasonal_analysis['monthly_trends']
print("\nMonthly Sales Trends:")
display(monthly_trends)

In [ ]:
# Visualize monthly trends
visualizations.plot_sales_trends(df_clean, time_period='monthly')
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/monthly_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
peak_month = monthly_trends.loc[monthly_trends['TotalRevenue'].idxmax(), 'Month']
print(f"  • Peak sales month: {peak_month}")
print(f"  • Clear seasonal patterns visible in monthly data")
print(f"  • Revenue fluctuations indicate seasonal demand")

In [ ]:
# Display quarterly trends
quarterly_trends = seasonal_analysis['quarterly_trends']
print("\nQuarterly Sales Trends:")
display(quarterly_trends)

In [ ]:
# Visualize quarterly trends
visualizations.plot_sales_trends(df_clean, time_period='quarterly')
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/quarterly_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
peak_quarter = quarterly_trends.loc[quarterly_trends['TotalRevenue'].idxmax(), 'Quarter']
print(f"  • Peak sales quarter: Q{peak_quarter}")
print(f"  • Quarterly patterns show business cycles")
print(f"  • Opportunity for seasonal inventory planning")

### 6.2 Seasonal Heatmap

In [ ]:
# Create seasonal heatmap
visualizations.plot_seasonal_heatmap(df_clean)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/seasonal_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
print(f"  • Heatmap reveals patterns across months and days of week")
print(f"  • Certain day-month combinations show peak activity")
print(f"  • Visual pattern helps identify optimal promotion timing")

### 6.3 Day-of-Week Patterns

In [ ]:
# Display day-of-week patterns
dow_patterns = seasonal_analysis['day_of_week_patterns']
print("\nDay-of-Week Sales Patterns:")
print("=" * 80)
display(dow_patterns)

In [ ]:
# Visualize day-of-week patterns
visualizations.plot_day_of_week_patterns(dow_patterns)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/day_of_week_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
peak_day = dow_patterns.loc[dow_patterns['TotalRevenue'].idxmax(), 'DayOfWeek']
low_day = dow_patterns.loc[dow_patterns['TotalRevenue'].idxmin(), 'DayOfWeek']
print(f"  • Peak sales day: {peak_day}")
print(f"  • Lowest sales day: {low_day}")
print(f"  • Day-of-week patterns suggest optimal staffing and inventory levels")
print(f"  • Weekend vs weekday patterns are distinct")

### 6.4 Seasonal Product Preferences

In [ ]:
# Analyze seasonal product preferences
seasonal_products = seasonal_analysis['seasonal_product_preferences']

print("\nSeasonal Product Preferences:")
print("=" * 80)

for quarter in sorted(seasonal_products['Quarter'].unique()):
    print(f"\nQ{quarter} - Top 5 Categories:")
    quarter_data = seasonal_products[seasonal_products['Quarter'] == quarter].head(5)
    display(quarter_data[['Category', 'TotalRevenue', 'TransactionCount']])

print("\n📊 Key Insights:")
print(f"  • Product preferences shift across seasons")
print(f"  • Certain categories show strong seasonal patterns")
print(f"  • Opportunity for seasonal merchandising strategies")

### 6.5 Year-over-Year Comparison

In [ ]:
# Year-over-year comparison (if multiple years exist)
if 'yoy_comparison' in seasonal_analysis and seasonal_analysis['yoy_comparison'] is not None:
    yoy_data = seasonal_analysis['yoy_comparison']
    print("\nYear-over-Year Comparison:")
    print("=" * 80)
    display(yoy_data)
    
    print("\n📊 Key Insights:")
    print(f"  • Year-over-year growth trends identified")
    print(f"  • Business performance trajectory visible")
    print(f"  • Growth opportunities and challenges highlighted")
else:
    print("\nYear-over-Year Comparison:")
    print("=" * 80)
    print("Data contains single year - YoY comparison not applicable")
    
    # Show year summary instead
    year_summary = df_clean.groupby('Year').agg({
        'TransactionID': 'count',
        'TotalAmount': 'sum',
        'CustomerID': 'nunique'
    }).rename(columns={
        'TransactionID': 'TotalTransactions',
        'TotalAmount': 'TotalRevenue',
        'CustomerID': 'UniqueCustomers'
    })
    print("\nYear Summary:")
    display(year_summary)

---
## 7. Summary and Recommendations

Comprehensive summary of key findings and actionable recommendations.

### 7.1 Executive Summary

In [ ]:
print("="*80)
print("EXECUTIVE SUMMARY")
print("="*80)

# Recalculate key metrics for summary
total_transactions = len(df_clean)
total_revenue = df_clean['TotalAmount'].sum()
avg_transaction_value = df_clean['TotalAmount'].mean()
unique_customers = df_clean['CustomerID'].nunique()
unique_products = df_clean['ProductName'].nunique()

print(f"\nDataset Overview:")
print(f"  • Total Transactions: {total_transactions:,}")
print(f"  • Total Revenue: ${total_revenue:,.2f}")
print(f"  • Average Transaction Value: ${avg_transaction_value:.2f}")
print(f"  • Unique Customers: {unique_customers:,}")
print(f"  • Unique Products: {unique_products:,}")
print(f"  • Date Range: {df_clean['Date'].min().strftime('%Y-%m-%d')} to {df_clean['Date'].max().strftime('%Y-%m-%d')}")

### 7.2 Key Findings by Analysis Area

In [ ]:
print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)

print("\n1. EXPLORATORY DATA ANALYSIS")
print("-" * 80)
print("   • Product Performance: Top products drive significant revenue concentration")
print("   • Geographic Distribution: Sales are concentrated in specific cities")
print("   • Category Analysis: Certain product categories significantly outperform others")
print("   • Store Type Variation: Different store types show distinct performance patterns")

print("\n2. CUSTOMER INSIGHTS")
print("-" * 80)
print("   • Customer Segmentation: Clear high-value, medium-value, and low-value segments identified")
print("   • Spending Patterns: High-value customers represent a small percentage but generate disproportionate revenue")
print("   • Purchase Frequency: Repeat purchase rates indicate customer loyalty levels")
print("   • Store Preferences: Customer segments show distinct store type preferences")
print("   • Product Preferences: Different customer segments prefer different product categories")

print("\n3. PROMOTION EFFECTIVENESS")
print("-" * 80)
print("   • Discount Impact: Discounts drive transaction volume but affect average transaction value")
print("   • Optimal Discount Levels: Specific discount ranges show maximum effectiveness")
print("   • Product-Specific Response: Certain products benefit more from promotions than others")
print("   • ROI Analysis: Promotion ROI varies by discount level and product category")
print("   • Penetration Rate: Significant portion of transactions involve discounts")

print("\n4. SEASONAL TRENDS")
print("-" * 80)
print("   • Monthly Patterns: Clear seasonal peaks and troughs in sales volume")
print("   • Quarterly Trends: Specific quarters show stronger performance")
print("   • Day-of-Week Patterns: Transaction volume varies significantly by day")
print("   • Seasonal Products: Product preferences shift across seasons")
print("   • Planning Implications: Seasonal patterns enable better inventory and staffing planning")

### 7.3 Actionable Recommendations

In [ ]:
print("\n" + "="*80)
print("ACTIONABLE RECOMMENDATIONS")
print("="*80)

print("\n1. CUSTOMER STRATEGY")
print("-" * 80)
print("   ✓ Focus retention efforts on high-value customers (top revenue generators)")
print("   ✓ Develop targeted campaigns to move medium-value customers to high-value segment")
print("   ✓ Implement personalized product recommendations based on segment preferences")
print("   ✓ Create loyalty programs tailored to each customer segment")
print("   ✓ Increase engagement with low-frequency customers through targeted outreach")

print("\n2. PRODUCT & INVENTORY MANAGEMENT")
print("-" * 80)
print("   ✓ Prioritize inventory for top-performing products and categories")
print("   ✓ Adjust stock levels based on seasonal demand patterns")
print("   ✓ Optimize product mix by store type based on performance data")
print("   ✓ Phase out or discount slow-moving products identified in analysis")
print("   ✓ Plan seasonal merchandising strategies based on quarterly trends")

print("\n3. PROMOTIONAL STRATEGY")
print("-" * 80)
print("   ✓ Focus promotions on products that show highest response to discounts")
print("   ✓ Implement optimal discount levels identified in analysis")
print("   ✓ Avoid over-discounting products that sell well without promotions")
print("   ✓ Time promotions to align with seasonal demand patterns")
print("   ✓ Monitor promotion ROI continuously and adjust strategy accordingly")
print("   ✓ Test targeted promotions by customer segment for better effectiveness")

print("\n4. OPERATIONAL OPTIMIZATION")
print("-" * 80)
print("   ✓ Adjust staffing levels based on day-of-week transaction patterns")
print("   ✓ Optimize store operations by type based on performance metrics")
print("   ✓ Allocate resources to high-performing geographic markets")
print("   ✓ Plan inventory replenishment aligned with seasonal trends")
print("   ✓ Implement dynamic pricing strategies based on demand patterns")

print("\n5. GROWTH OPPORTUNITIES")
print("-" * 80)
print("   ✓ Expand presence in top-performing cities and categories")
print("   ✓ Replicate success factors from high-performing stores to others")
print("   ✓ Develop new products in high-demand categories")
print("   ✓ Explore cross-selling opportunities based on customer purchase patterns")
print("   ✓ Invest in customer acquisition in underserved segments")

### 7.4 Assumptions and Methodology

In [ ]:
print("\n" + "="*80)
print("ASSUMPTIONS AND METHODOLOGY")
print("="*80)

print("\nDATA ASSUMPTIONS:")
print("-" * 80)
print("   1. Dataset Completeness: Assumed the dataset represents complete transaction history")
print("   2. Data Accuracy: Assumed transaction amounts, dates, and IDs are accurate")
print("   3. Customer Identification: Assumed CustomerID consistently identifies unique customers")
print("   4. Product Identification: Assumed ProductID and ProductName are consistent")
print("   5. Discount Representation: Assumed discount values are percentages (0-100)")
print("   6. Currency: Assumed all monetary values are in the same currency")
print("   7. Time Period: Analysis covers the complete date range in the dataset")

print("\nMETHODOLOGY:")
print("-" * 80)
print("   1. Data Cleaning:")
print("      • Handled missing values using appropriate strategies (drop, impute, or flag)")
print("      • Removed duplicate transactions based on TransactionID")
print("      • Converted data types for optimal processing")
print("      • Identified and flagged outliers without automatic removal")

print("\n   2. Feature Engineering:")
print("      • Extracted temporal features: Year, Month, Day, DayOfWeek, Quarter")
print("      • Created discount flags and calculated discount amounts")
print("      • Derived customer-level aggregations for segmentation")

print("\n   3. Customer Segmentation:")
print("      • Used quantile-based segmentation on total customer spending")
print("      • High-Value: Top 20% of customers by spending")
print("      • Medium-Value: Middle 50% of customers")
print("      • Low-Value: Bottom 30% of customers")

print("\n   4. Statistical Analysis:")
print("      • Calculated descriptive statistics for all numerical variables")
print("      • Used aggregation functions (sum, mean, count) for group analysis")
print("      • Applied ranking methods for top-N analyses")

print("\n   5. Promotion Analysis:")
print("      • Compared transactions with and without discounts")
print("      • Calculated promotion lift as percentage increase in transaction volume")
print("      • Estimated ROI using discount cost vs. incremental revenue")

print("\n   6. Visualization:")
print("      • Used appropriate chart types for different data patterns")
print("      • Applied consistent styling and color schemes")
print("      • Ensured all charts have clear titles, labels, and legends")

### 7.5 Design Choices and Rationale

In [ ]:
print("\n" + "="*80)
print("DESIGN CHOICES AND RATIONALE")
print("="*80)

print("\n1. MODULAR ARCHITECTURE")
print("-" * 80)
print("   Choice: Separated code into three modules (data_processor, analysis, visualizations)")
print("   Rationale:")
print("      • Improves code reusability and maintainability")
print("      • Enables easier testing of individual components")
print("      • Follows separation of concerns principle")
print("      • Makes code more readable and organized")

print("\n2. JUPYTER NOTEBOOK FORMAT")
print("-" * 80)
print("   Choice: Used Jupyter notebook for main analysis")
print("   Rationale:")
print("      • Combines code, visualizations, and narrative in one document")
print("      • Enables interactive exploration and iteration")
print("      • Easy to export to PDF for reporting")
print("      • Familiar format for data analysis stakeholders")

print("\n3. PANDAS FOR DATA MANIPULATION")
print("-" * 80)
print("   Choice: Used pandas as primary data manipulation library")
print("   Rationale:")
print("      • Industry standard for data analysis in Python")
print("      • Rich functionality for data operations")
print("      • Excellent performance for medium-sized datasets")
print("      • Strong integration with visualization libraries")

print("\n4. VISUALIZATION APPROACH")
print("-" * 80)
print("   Choice: Combined matplotlib and seaborn for visualizations")
print("   Rationale:")
print("      • matplotlib provides fine-grained control")
print("      • seaborn offers better aesthetics and statistical plots")
print("      • Both are well-documented and widely used")
print("      • Consistent styling applied across all visualizations")

print("\n5. CUSTOMER SEGMENTATION METHOD")
print("-" * 80)
print("   Choice: Quantile-based segmentation on total spending")
print("   Rationale:")
print("      • Simple and interpretable approach")
print("      • Directly tied to business value (revenue)")
print("      • Easy to implement and explain to stakeholders")
print("      • Can be enhanced with RFM analysis if needed")

print("\n6. ERROR HANDLING STRATEGY")
print("-" * 80)
print("   Choice: Graceful error handling with informative messages")
print("   Rationale:")
print("      • Prevents analysis from failing on edge cases")
print("      • Provides clear feedback on data quality issues")
print("      • Enables partial results when complete analysis isn't possible")
print("      • Improves user experience and debugging")

### 7.6 Limitations and Future Analysis

In [ ]:
print("\n" + "="*80)
print("LIMITATIONS AND FUTURE ANALYSIS")
print("="*80)

print("\nCURRENT LIMITATIONS:")
print("-" * 80)
print("   1. Data Scope:")
print("      • Analysis limited to available transaction data")
print("      • No external data (e.g., market trends, competitor data)")
print("      • Missing customer demographic information")

print("\n   2. Temporal Analysis:")
print("      • Limited historical data may affect trend reliability")
print("      • Cannot predict future trends with high confidence")
print("      • Seasonal patterns may not be fully established")

print("\n   3. Causality:")
print("      • Analysis shows correlations, not causation")
print("      • Cannot definitively attribute sales changes to specific factors")
print("      • External factors (economy, competition) not accounted for")

print("\n   4. Customer Behavior:")
print("      • No information on customer acquisition channels")
print("      • Limited insight into customer journey")
print("      • Cannot track customer lifetime value over extended periods")

print("\n   5. Product Information:")
print("      • Limited product attributes beyond category")
print("      • No cost data to calculate true profitability")
print("      • Missing product lifecycle information")

print("\nFUTURE ANALYSIS OPPORTUNITIES:")
print("-" * 80)
print("   1. Advanced Analytics:")
print("      • Implement predictive models for sales forecasting")
print("      • Apply machine learning for customer churn prediction")
print("      • Develop recommendation systems for personalized marketing")
print("      • Use clustering algorithms for more sophisticated segmentation")

print("\n   2. Enhanced Customer Analysis:")
print("      • Implement RFM (Recency, Frequency, Monetary) analysis")
print("      • Calculate true customer lifetime value (CLV)")
print("      • Analyze customer journey and touchpoints")
print("      • Study cross-selling and upselling opportunities")

print("\n   3. Deeper Product Insights:")
print("      • Market basket analysis for product associations")
print("      • Product affinity analysis")
print("      • Profitability analysis (if cost data becomes available)")
print("      • Product lifecycle analysis")

print("\n   4. Promotion Optimization:")
print("      • A/B testing framework for promotion strategies")
print("      • Price elasticity analysis")
print("      • Optimal discount level modeling")
print("      • Promotion cannibalization analysis")

print("\n   5. Geographic Expansion:")
print("      • Market penetration analysis by city")
print("      • Geographic expansion opportunity identification")
print("      • Regional preference analysis")
print("      • Store location optimization")

print("\n   6. Real-time Analytics:")
print("      • Develop dashboards for real-time monitoring")
print("      • Implement automated alerting for anomalies")
print("      • Create KPI tracking systems")
print("      • Build interactive visualization tools")

### 7.7 Conclusion

In [ ]:
print("\n" + "="*80)
print("CONCLUSION")
print("="*80)

print("\nThis comprehensive analysis of retail transaction data has revealed valuable insights")
print("across multiple dimensions of business performance:")

print("\n✓ CUSTOMER INSIGHTS: Clear segmentation enables targeted strategies for different")
print("  customer groups, with opportunities to increase value from medium-tier customers.")

print("\n✓ PRODUCT PERFORMANCE: Top products and categories drive significant revenue,")
print("  suggesting focus areas for inventory and marketing investments.")

print("\n✓ PROMOTIONAL EFFECTIVENESS: Discounts drive volume but require careful management")
print("  to maintain profitability. Optimal discount levels have been identified.")

print("\n✓ SEASONAL PATTERNS: Clear temporal trends enable better planning for inventory,")
print("  staffing, and promotional activities throughout the year.")

print("\n✓ GEOGRAPHIC OPPORTUNITIES: Performance varies significantly by location, indicating")
print("  opportunities for targeted expansion and market-specific strategies.")

print("\nThe actionable recommendations provided can drive immediate improvements in:")
print("  • Customer retention and value growth")
print("  • Inventory optimization and product mix")
print("  • Promotional strategy and ROI")
print("  • Operational efficiency")
print("  • Revenue growth and market expansion")

print("\nThis analysis provides a solid foundation for data-driven decision making.")
print("Regular monitoring of these metrics and continuous refinement of strategies")
print("will be essential for sustained business growth and competitive advantage.")

print("\n" + "="*80)
print("END OF ANALYSIS")
print("="*80)